In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from load_data import read_load_history, read_temperature_history, normalization, de_normalization

Using Theano backend.


In [2]:
print('Loading Data')
length_temperature, time_temperature, temperature = read_temperature_history()
length_load, time_load, load = read_load_history(if_full=True)

Loading Data


In [3]:
temperature = temperature.reshape((len(temperature),11))
time_temperature = time_temperature.reshape((len(time_temperature),4))
load = load.reshape((len(load),20))

In [4]:
t = temperature[0:39414]
tt = time_temperature[0:39414]
l = load[0:39414]

In [5]:
# uses history average to fill the missing data
step_size = 365 * 24
for i in range(168+18):
    index  = len(t)
    temp = np.zeros((1,11))
    time = np.zeros((1,4))
    time  = np.add(time,tt[index - step_size - 24])
    time[0][0] += 1
    load_temp = np.zeros((1,20))
    load_temp = np.add(load_temp,l[index - 7*24])
    l = np.append(l,load_temp,axis=0)
    tt = np.append(tt,time,axis=0)
    for j in range(1,5):
        previous_index = index - j*step_size - 24
        previous_temp = t[previous_index]
        temp = np.add(temp,previous_temp)
    temp = temp/4
    t = np.append(t,temp,axis=0)

In [6]:
import pandas as pd
df_t = pd.DataFrame(t)
df_t.to_csv('./data/full_temp.csv', header=False, index=False)
df_tt = pd.DataFrame(tt)
df_tt.to_csv('./data/full_temp_time.csv', header=False, index=False)
df_l = pd.DataFrame(l)
df_l.to_csv('./data/full_load.csv', header=False, index=False)

In [22]:
# baseline uses previous years' load history average
t = temperature[0:39414 - 168]
tt = time_temperature[0:39414 - 168]
l = load[0:39414-168]
step_size = 365 * 24
for i in range(168):
    index  = len(t)
    load_temp = np.zeros((1,20))
    for j in range(1,5):
        previous_index = index - j*step_size - 24
        previous_temp = l[previous_index]
        load_temp = np.add(load_temp,previous_temp)
    load_temp = load_temp/4
    l = np.append(l,load_temp,axis=0)

In [23]:
# baseline uses previous years' load history average
print(len(l))
from load_data import read_benchmark
import pandas as pd
predicted_output = l
benchmark = load[0:39414][-1-168:-1]
from sklearn.metrics import mean_squared_error
pred_future = predicted_output[-1-168:-1]
temp = np.zeros((len(pred_future),1))
benchmark_temp = np.zeros((len(benchmark),1))
for i in range(len(pred_future)):
    temp[i][0] = np.sum(pred_future[i])
    benchmark_temp[i][0] = np.sum(benchmark[i])
pred_future = np.concatenate((pred_future,temp),axis=1)
benchmark = np.concatenate((benchmark,benchmark_temp),axis=1)
rsme = np.sqrt(mean_squared_error(benchmark, pred_future))
print('rsme:',rsme)

39414
rsme: 167462.797639


In [24]:
# baseline uses previous week's load as prediction 
t = temperature[0:39414 - 168]
tt = time_temperature[0:39414 - 168]
l = load[0:39414 - 168]
step_size = 7 * 24
for i in range(168):
    index  = len(t)
    load_temp = np.zeros((1,20))
    previous_index = index - j*step_size
    previous_temp = l[previous_index]
    load_temp = np.add(load_temp,previous_temp)
    l = np.append(l,load_temp,axis=0)

In [25]:
# baseline uses previous week's load as prediction 
print(len(l))
from load_data import read_benchmark
import pandas as pd
predicted_output = l
benchmark = load[0:39414][-1-168:-1]
from sklearn.metrics import mean_squared_error
pred_future = predicted_output[-1-168:-1]
temp = np.zeros((len(pred_future),1))
benchmark_temp = np.zeros((len(benchmark),1))
for i in range(len(pred_future)):
    temp[i][0] = np.sum(pred_future[i])
    benchmark_temp[i][0] = np.sum(benchmark[i])
pred_future = np.concatenate((pred_future,temp),axis=1)
benchmark = np.concatenate((benchmark,benchmark_temp),axis=1)
rsme = np.sqrt(mean_squared_error(benchmark, pred_future))
print('rsme:',rsme)

39414
rsme: 227129.421051
